In [81]:
import transformers
print(transformers.__version__)

4.30.2


In [1]:
import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import torch
import json
import gc
import re

from sklearn.model_selection import train_test_split

import re
from tqdm import tqdm

2023-09-25 11:31:42.223735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 11:31:44.019386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import pandas as pd
output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)
df['encoded_label'] = df['label'].astype('category').cat.codes
id2label = {label:category for category, label in sorted(df[['label', 'encoded_label']].drop_duplicates().values, key = lambda x:x[1])}
label2id = {category:label for category, label in sorted(df[['label', 'encoded_label']].drop_duplicates().values, key = lambda x:x[1])}



In [6]:
output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)


df['encoded_label'] = df['label'].astype('category').cat.codes
data_texts = df['text'].tolist()
data_labels = df['encoded_label'].tolist()
NUM_LABELS = len(df['encoded_label'].drop_duplicates())
df.info()
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)
print(len(train_texts), len(val_texts))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60835 entries, 0 to 60834
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           60835 non-null  object
 1   label          60835 non-null  object
 2   encoded_label  60835 non-null  int8  
dtypes: int8(1), object(2)
memory usage: 1010.1+ KB
48668 12167


In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)
samples_per_category = 100
selected_data = pd.DataFrame(columns=df.columns)

unique_categories = df['label'].unique()

for category in unique_categories:

    category_samples = df[df['label'] == category]
    
    if len(category_samples) >= samples_per_category:
        selected_samples = category_samples.sample(n=samples_per_category, random_state=42)
    else:
        selected_samples = category_samples
    
    selected_data = selected_data.append(selected_samples)

selected_data = selected_data.reset_index(drop=True)

selected_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100 entries, 0 to 8099
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8100 non-null   object
 1   label   8100 non-null   object
dtypes: object(2)
memory usage: 126.7+ KB


In [2]:
import pandas as pd
import numpy as np
import torch
import logging  # Import the logging module
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(filename='/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_30/training.log', level=logging.INFO)  # Specify the log file

# Define your dataset (selected_data) here

# selected_data['encoded_label'] = selected_data['label'].astype('category').cat.codes
# data_texts = selected_data['text'].tolist()
# data_labels = selected_data['encoded_label'].tolist()
# NUM_LABELS = len(selected_data['encoded_label'].drop_duplicates())

output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)


df['encoded_label'] = df['label'].astype('category').cat.codes
data_texts = df['text'].tolist()
data_labels = df['encoded_label'].tolist()
NUM_LABELS = len(df['encoded_label'].drop_duplicates())

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)

model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    
    # Calculate F1 score for multi-class classification (macro-average)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1['f1']
    }

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir ='./results',
    num_train_epochs=30,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,  # Add this line to specify the logging frequency
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Log the training output
for line in trainer.state.log_history:
    logger.info(line)

results = trainer.evaluate()

model.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_30')
tokenizer.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_30')

print("--------------- MODEL OF 30 OF EPOCH HAS BEEN TRAINED ---------------")

import pandas as pd
import numpy as np
import torch
import logging  # Import the logging module
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(filename='/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20/training.log', level=logging.INFO)  # Specify the log file

output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)


df['encoded_label'] = df['label'].astype('category').cat.codes
data_texts = df['text'].tolist()
data_labels = df['encoded_label'].tolist()
NUM_LABELS = len(df['encoded_label'].drop_duplicates())

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)

model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    
    # Calculate F1 score for multi-class classification (macro-average)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1['f1']
    }

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir ='./results',
    num_train_epochs=20,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=200,  # Add this line to specify the logging frequency
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Log the training output
for line in trainer.state.log_history:
    logger.info(line)

results = trainer.evaluate()

model.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20')
tokenizer.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20')

print("--------------- MODEL OF 20 OF EPOCH HAS BEEN TRAINED ---------------")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.

Step,Training Loss,Validation Loss,Accuracy,F1
500,3.258300,1.579829,0.499630,0.358922
1000,1.046800,0.691090,0.745295,0.651333
1500,0.611200,0.507922,0.810882,0.762222
2000,0.484500,0.441784,0.834635,0.784508
2500,0.455700,0.444827,0.839155,0.795986
3000,0.410900,0.396091,0.855264,0.825988
3500,0.362600,0.388714,0.866771,0.838228
4000,0.350700,0.391988,0.861182,0.834679
4500,0.352300,0.349344,0.876140,0.858332
5000,0.337300,0.354841,0.872688,0.848265


--------------- MODEL OF 30 OF EPOCH HAS BEEN TRAINED ---------------


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.

ValueError: --load_best_model_at_end requires the saving steps to be a round multiple of the evaluation steps, but found 500, which is not a round multiple of 200.

In [ ]:
import pandas as pd
import numpy as np
import torch
import logging  # Import the logging module
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(filename='/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20/training.log', level=logging.INFO)  # Specify the log file

output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)


df['encoded_label'] = df['label'].astype('category').cat.codes
data_texts = df['text'].tolist()
data_labels = df['encoded_label'].tolist()
NUM_LABELS = len(df['encoded_label'].drop_duplicates())

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)

model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    
    # Calculate F1 score for multi-class classification (macro-average)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1['f1']
    }

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir ='./results',
    num_train_epochs=20,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=200,  # Add this line to specify the logging frequency
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Log the training output
for line in trainer.state.log_history:
    logger.info(line)

results = trainer.evaluate()

model.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20')
tokenizer.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT_20')

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
# df = pd.read_csv(output_dir)


# df['encoded_label'] = df['label'].astype('category').cat.codes
# data_texts = df['text'].tolist()
# data_labels = df['encoded_label'].tolist()
# NUM_LABELS = len(df['encoded_label'].drop_duplicates())

selected_data['encoded_label'] = selected_data['label'].astype('category').cat.codes
data_texts = selected_data['text'].tolist()
data_labels = selected_data['encoded_label'].tolist()
NUM_LABELS = len(selected_data['encoded_label'].drop_duplicates())

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)

model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
print(train_encodings['input_ids'].shape)

val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
print(val_encodings['input_ids'].shape)

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

# metric = load_metric("accuracy")
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    
    # Calculate F1 score for multi-class classification (macro-average)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    
    return {
        "accuracy": accuracy,
        "f1": f1['f1']
    }

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir ='./results',
    num_train_epochs = 1,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps = 500,
    weight_decay = 1e-5,
    logging_dir = './logs',
    evaluation_strategy="steps",
    eval_steps = 500,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
results = trainer.evaluate()

model.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT')
tokenizer.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT')

2023-09-21 20:05:28.771629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 20:05:29.569125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Dis

torch.Size([6480, 512])
torch.Size([1620, 512])


/tmp/ipykernel_68040/224354159.py:59: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/home/wipro/NLP_ENV/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


Trainer is attempting to log a value of "{'accuracy': 0.3771604938271605}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/tokenizer_config.json',
 '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/special_tokens_map.json',
 '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/vocab.txt',
 '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/added_tokens.json',
 '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/tokenizer.json')

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)


df['encoded_label'] = df['label'].astype('category').cat.codes
data_texts = df['text'].tolist()
data_labels = df['encoded_label'].tolist()
NUM_LABELS = len(df['encoded_label'].drop_duplicates())

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)

model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

# Tokenize the data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
print(train_encodings['input_ids'].shape)

val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
print(val_encodings['input_ids'].shape)

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings.input_ids[idx],
            'attention_mask': self.encodings.attention_mask[idx],
            'labels': torch.tensor(self.labels[idx])
        }
        return item

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir ='./results',
    num_train_epochs = 1,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps = 500,
    weight_decay = 1e-5,
    logging_dir = './logs',
    evaluation_strategy="steps",
    eval_steps = 500,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
results = trainer.evaluate()

model.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT')
tokenizer.save_pretrained('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT')

In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT'  
tokenizer_path = '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT' 

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

df = pd.read_excel('/home/wipro/NLP_RnD/PB_EC/EDA/AI_Email_Classification_Dump_04_09_23.xlsx', engine = 'openpyxl')
df = df[['text']]
text_to_classify = df['text'].tolist()
for text in text_to_classify:
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors='pt')

    with torch.no_grad():
        logits = model(**inputs).logits
    
    probabilities = torch.softmax(logits, dim=1)
    
    predicted_class = torch.argmax(probabilities, dim=1).item()
    
    confidence_score = probabilities.max(dim=1).values.item()
    
    predicted_label = id2label[predicted_class]
    
    print(f"Predicted Label: {predicted_label}")
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence Score: {confidence_score}")


Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.9900088310241699
Predicted Label: Pay Later_Eligiblity
Predicted Class: 65
Confidence Score: 0.2783282995223999
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.9610448479652405
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.3593064844608307
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.6442752480506897
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.9771851897239685
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.8627548217773438
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.8627548217773438
Predicted Label: Pay Later_Paylater Bill Generation
Predicted Class: 66
Confidence Score: 0.8627548217773438
Predicted Label: Pay Later_Paylat

KeyboardInterrupt: 

## CODE TO PREDICT THE MODEL 

In [5]:
import pickle
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT'  
tokenizer_path = '/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT' 

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
   
with open('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/id2label.pickle', 'rb') as file:
    id2label = pickle.load(file)

df = pd.read_excel('/home/wipro/NLP_RnD/PB_EC/EDA/AI_Email_Classification_Dump_04_09_23.xlsx', engine='openpyxl')
df = df[['text']]
text_to_classify = df['text'].tolist()

predicted_labels = []
predicted_classes = []
confidence_scores = []
for text in text_to_classify:
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence_score = probabilities.max(dim=1).values.item()
    predicted_label = id2label[predicted_class]
    
    predicted_labels.append(predicted_label)
    predicted_classes.append(predicted_class)
    confidence_scores.append(confidence_score)

results_df = pd.DataFrame({
    'Text': text_to_classify,
    'Predicted Label': predicted_labels,
    'Predicted Class': predicted_classes,
    'Confidence Score': confidence_scores
})


results_df.to_excel('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/OUTPUT/Predicted_Output.xlsx', index=False)


In [ ]:
def predict_output(text, model, tokenizer, id2label):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence_score = probabilities.max(dim=1).values.item()
    predicted_label = id2label[predicted_class]

    ec_res = {'class_': predicted_label, 'confidence': confidence_score}
    return ec_res

In [47]:
import pandas as pd


dff = pd.read_excel('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/Validation_results_22_09_2023_25_cats_test_22Dec.xlsx', engine='openpyxl')
output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)

label_counts = df['label'].value_counts().to_dict()

for index, row in dff.iterrows():
    sub_category = row['Sub-Category']
    
    if sub_category in label_counts:
        count = label_counts[sub_category]
        dff.at[index, 'Sub-Category'] = {sub_category: count}

dff.to_excel('/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/Updated_Validation_results.xlsx', index=False, engine='openpyxl')

In [19]:
import pandas as pd
output_dir = "/home/wipro/NLP_RnD/PB_EC/EDA/DISTILLBERT/81_subcats_pb_ec_train_06092023.csv"
df = pd.read_csv(output_dir)
counts = df.label.value_counts().tolist()
labels = df.label.value_counts().index
value_count = df.label.value_counts()

In [24]:
label2count = {"label":labels,
               "Count ":counts}
label2count = {"label": labels, "Count": counts}
result = [{"label": label, "Count": count} for label, count in zip(label2count["label"], label2count["Count"])]

# Example usage:
print(result)


[{'label': 'Imobile_Imobile Technical Issue SR', 'Count': 900}, {'label': 'Account Modification_Change Of Communication Address', 'Count': 900}, {'label': 'FD closure procedure', 'Count': 900}, {'label': 'Charges Related - RL_Mab Savings Account General Inquiry', 'Count': 900}, {'label': 'Account Closure_Account closure procedure', 'Count': 900}, {'label': 'Account Closure_Account closure reason probing', 'Count': 900}, {'label': 'Transaction Status_Transaction Refunded By The Merchant', 'Count': 900}, {'label': 'Customer want to apply and enq Eligibility criteria_', 'Count': 900}, {'label': 'Internet Banking login - RL_Know USER ID/Activation of USER ID', 'Count': 900}, {'label': 'PROCEDURE FOR REMOVAL OF FREEZE', 'Count': 900}, {'label': 'Charges Related - RL_Cash Transaction Charges Savings Account', 'Count': 900}, {'label': 'Credit Card Status_Redispatch Of The Card', 'Count': 900}, {'label': 'Credit Card Status_Apin Generation Process', 'Count': 900}, {'label': 'Charges Related - 